In [189]:
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import numpy as np

In [159]:
df =pd.read_pickle("evaluation_df.pkl").sort_values(by="prediction_time")
df['prediction_day'] = [str(i)[0:10] for i in df['prediction_time']]
df['prediction_day'] = [str(i)[0:10] for i in df['prediction_time']]


In [162]:
df

,station,prediction_time,prediction_window,number_of_bikes_left,number_of_bikes_returned,net_difference,prediction_day
3353,81,2015-01-16 07:45:00,15,0,0,0,2015-01-16
8616,416,2015-01-16 07:45:00,15,0,0,0,2015-01-16
5488,108,2015-01-16 07:45:00,15,0,0,0,2015-01-16
1667,98,2015-01-16 07:45:00,15,0,0,0,2015-01-16
9891,68,2015-01-16 07:45:00,15,0,0,0,2015-01-16
...,...,...,...,...,...,...,...
2593,59,2022-08-31 21:30:00,15,3,2,-1,2022-08-31
4936,16,2022-08-31 21:30:00,15,2,2,0,2022-08-31
6971,85,2022-08-31 21:30:00,15,1,1,0,2022-08-31
4752,72,2022-08-31 21:30:00,15,3,2,-1,2022-08-31


In [169]:
mean_values = df.groupby([df.prediction_time.dt.weekday,df.prediction_time.dt.hour]).mean()
print(len(mean_values),"combos")
mean_dict = mean_values['net_difference'].to_dict()
zips = list(zip(df.prediction_time.dt.weekday,df.prediction_time.dt.hour))
preds = [mean_dict.get(i) for i in zips]
mean_squared_error(df['net_difference'], preds, squared=False)

157 combos


1.6708128312330772

In [168]:
#controlling for station / hr 
mean_values = df.groupby(['station',df.prediction_time.dt.hour]).mean()
print(len(mean_values),"combos")
mean_dict = mean_values['net_difference'].to_dict()
zips = list(zip(df['station'],df.prediction_time.dt.hour))
preds = [mean_dict.get(i) for i in zips]
mean_squared_error(df['net_difference'], preds, squared=False)

3691 combos


1.4031538415793092

In [167]:
#gets even better!!! 
#controlling for station / week / hr 
mean_values = df.groupby(['station',df.prediction_time.dt.weekday,df.prediction_time.dt.hour]).mean()
print(len(mean_values),"combos")
mean_dict = mean_values['net_difference'].to_dict()
zips = list(zip(df['station'],df.prediction_time.dt.weekday,df.prediction_time.dt.hour))
preds = [mean_dict.get(i) for i in zips]
mean_squared_error(df['net_difference'], preds, squared=False)

7708 combos


0.8790884528513089

In [166]:
#controlling for station / month / weekday / hr 
mean_values = df.groupby(['station',df.prediction_time.dt.month,df.prediction_time.dt.weekday,df.prediction_time.dt.hour]).mean()
print(len(mean_values),"combos")
mean_dict = mean_values['net_difference'].to_dict()
zips = list(zip(df['station'],df.prediction_time.dt.month,df.prediction_time.dt.weekday,df.prediction_time.dt.hour))
preds = [mean_dict.get(i) for i in zips]
mean_squared_error(df['net_difference'], preds, squared=False)

9451 combos


0.33236526092037155

In [176]:
df2 = pd.read_csv("Boston - Weather Data - Adj.csv")

In [211]:
df['month'] = df.prediction_time.dt.month
df['hr'] = df.prediction_time.dt.hour
weather_df = df.merge(df2,how='left', left_on='prediction_day', right_on='DATE')
weather_df
x = weather_df[['Average Wind','Preciptiation','Snow','Temperature', 'month', 'hr']]
x

TypeError: data type 'categorical' not understood

In [208]:
from sklearn.model_selection import train_test_split
# splitting the data
x_train, x_test, y_train, y_test = train_test_split(x, weather_df['net_difference'], test_size = 0.2, random_state = 42)


In [209]:
model = LinearRegression()
model.fit(x_train,y_train)


LinearRegression()

In [210]:
y_prediction =  model.predict(x_test)
y_prediction


array([-0.0287087 , -0.01843999,  0.06427061, ..., -0.0731807 ,
        0.02774331, -0.06151099])

In [206]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)

print('r2 score', score)
print('mean_sqrd_error is==',mean_squared_error(y_test,y_prediction))
print('root_mean_squared error of is==',np.sqrt(mean_squared_error(y_test,y_prediction)))
#pretty bad with just weather... 

r2 score -0.002162500278071633
mean_sqrd_error is== 2.873118713134709
root_mean_squared error of is== 1.6950276437671183
